# Perfilamiento de código en R

Revisar las notas de Tere: [4.4 Rendimiento en R | Estadística Computacional](https://tereom.github.io/est-computacional-2019/rendimiento-en-r.html) para analizar el uso de recursos computacionales en los métodos de integración del rectángulo, trapecio y Simpson de [1.5.Integracion_numerica](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.5.Integracion_numerica.ipynb).

También podemos medir tiempos con el paquete [tictoc](https://www.rdocumentation.org/packages/tictoc/versions/1.0):

In [1]:
install.packages("tictoc",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/",verbose=TRUE)

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

foundpkgs: tictoc, /tmp/Rtmp7v2pNO/downloaded_packages/tictoc_1.0.tar.gz

files: /tmp/Rtmp7v2pNO/downloaded_packages/tictoc_1.0.tar.gz

1): succeeded '/usr/lib/R/bin/R CMD INSTALL -l '/usr/local/lib/R/site-library' /tmp/Rtmp7v2pNO/downloaded_packages/tictoc_1.0.tar.gz'



**Ejemplo:**

In [3]:
library(tictoc)

### 3) Regla compuesta del rectángulo

In [4]:
f<-function(x)exp(-x**2)

In [5]:
a<-0
b<-1
n<-10**6
h_hat<-(b-a)/n

**Forma secuencial**

In [6]:
Rcf1<-function(f,a,b,n){
    #Compute numerical approximation using rectangle or mid-point method in 
    #an interval.
    #Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    #Args:
    #    f (function): function of integrand
    #    a (int): left point of interval
    #    b (int): right point of interval
    #    n (int): number of subintervals
    #Returns:
    #    Rcf (float)
    h_hat<-(b-a)/n
    sum_res<-0
    for(j in 0:(n-1)){
        x<-a+(j+1/2.0)*h_hat
        sum_res<-sum_res+f(x)
    }
    h_hat*sum_res
}

In [7]:
system.time(aprox<-Rcf1(f,a,b,n))

   user  system elapsed 
  0.502   0.003   0.516 

In [8]:
err_relativo<-function(aprox,obj)abs(aprox-obj)/abs(obj)

In [9]:
obj<-integrate(Vectorize(f),a,b) #en la documentación de integrate
                                 #se menciona que se utilice Vectorize

In [10]:
err_relativo(aprox,obj$value)

[1] 6.719397e-14

In [11]:
tic("medición de sleep + regla del trapecio secuencial")
tic("medición de tiempo de regla de trapecio secuencial con tictoc")
tic()
Rcf1(f,a,b,n)
toc()
Sys.sleep(1)
toc()

[1] 0.7468241

medici<U+00F3>n de tiempo de regla de trapecio secuencial con tictoc: 0.544 sec elapsed
medici<U+00F3>n de sleep + regla del trapecio secuencial: 1.55 sec elapsed


# Solución: Perfilen la regla de Simpson con R con las herramientas vistas en 1.6.Perfilamiento_R.ipynb.

Primero instalamos los paquetes necesarios:

In [2]:
install.packages("schoolmath",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/",verbose=TRUE)

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

foundpkgs: schoolmath, /tmp/Rtmp7v2pNO/downloaded_packages/schoolmath_0.4.tar.gz

files: /tmp/Rtmp7v2pNO/downloaded_packages/schoolmath_0.4.tar.gz

1): succeeded '/usr/lib/R/bin/R CMD INSTALL -l '/usr/local/lib/R/site-library' /tmp/Rtmp7v2pNO/downloaded_packages/schoolmath_0.4.tar.gz'



In [3]:
install.packages("pracma",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/",verbose=TRUE)

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

foundpkgs: pracma, /tmp/Rtmp7v2pNO/downloaded_packages/pracma_2.2.9.tar.gz

files: /tmp/Rtmp7v2pNO/downloaded_packages/pracma_2.2.9.tar.gz

1): succeeded '/usr/lib/R/bin/R CMD INSTALL -l '/usr/local/lib/R/site-library' /tmp/Rtmp7v2pNO/downloaded_packages/pracma_2.2.9.tar.gz'



In [4]:
install.packages("profvis",lib="/usr/local/lib/R/site-library/",
                repos="https://cran.itam.mx/",verbose=TRUE)

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

foundpkgs: profvis, /tmp/Rtmp7v2pNO/downloaded_packages/profvis_0.3.6.tar.gz

files: /tmp/Rtmp7v2pNO/downloaded_packages/profvis_0.3.6.tar.gz

1): succeeded '/usr/lib/R/bin/R CMD INSTALL -l '/usr/local/lib/R/site-library' /tmp/Rtmp7v2pNO/downloaded_packages/profvis_0.3.6.tar.gz'



Y ahora los cargamos:

In [6]:
library(tictoc)
library(schoolmath)
library(pracma)
library(parallel)
library(profvis)

Definimos la función, los límites y número de subintervalos:

In [16]:
f<-function(x)exp(-x**2)

In [17]:
a<-0
b<-1
n<-10**5

Consideramos una primera versión de la regla de Simpson 

Notar que en esta primera opción sólo se usaron $n=10^5$ subintervalos porque con $n=10^6$ la función se tuvo que interrumpir por el tiempo que tomaba la ejecución

In [18]:
Scf<-function(n,f,a,b){
    #Compute numerical approximation using composed Simpson rule in 
    #an interval.
    #Nodes are generated via linspace
    #Args:
       # f (function): function of integrand
       # a (int): left point of interval
       # b (int): right point of interval
       # n (int): number of subintervals
    #Returns:
        #Scf (float)
   h<-(b-a)
   nodes1<-linspace(a,b,n+1)
   nodes<-nodes1[2:(length(nodes1)-1)]
   sum_res_par<-sum(f(nodes[is.even(seq_along(nodes))]))
   sum_res_impar<-sum(f(nodes[is.odd(seq_along(nodes))]))
   h/(3*n)*(f(nodes1[1])+f(nodes1[length(nodes1)])+2*sum_res_par+4*sum_res_impar)
}

Ahora veremos con la función system.time los tiempos en segundos que toma ejecutar la expresión:

**user time:** Tiempo usado por el CPU(s) para evaluar esta expresión, tiempo que experimenta la computadora.

**elapsed time:** tiempo en el reloj, tiempo que experimenta la persona.

In [19]:
system.time(aprox<-Scf(n,f,a,b))

   user  system elapsed 
 23.058   0.156  23.903 

Con esto se puede observar que el tiempo usado por el CPU fue de aprox 23 segundos y el tiempo en reloj fue de 24.

Ahora calculamos el error relativo de nuestra aproximación:

In [20]:
err_relativo<-function(aprox,obj)abs(aprox-obj)/abs(obj)

In [21]:
obj<-integrate(Vectorize(f),a,b)

In [22]:
err_relativo(aprox,obj$value)

[1] 2.973185e-16

In [23]:
tic("medición de sleep + regla de Simpson")
tic("medición de tiempo de regla de Simpson con tictoc")
tic()
Scf(n,f,a,b)
toc()
Sys.sleep(1)
toc()

[1] 0.7468241

elapsed time is 23.475000 seconds 
elapsed time is 24.480000 seconds 


Ahora usaremos la función profvis() del paquete con el mismo nombre que puede ser más útil cuando uno desconoce cuál es la función que alenta un programa

In [24]:
profvis({h<-(b-a)
        nodes1<-linspace(a,b,n+1)
        nodes<-nodes1[2:(length(nodes1)-1)]
        sum_res_par<-sum(f(nodes[is.even(seq_along(nodes))]))
        sum_res_impar<-sum(f(nodes[is.odd(seq_along(nodes))]))
        h/(3*n)*(f(nodes1[1])+f(nodes1[length(nodes1)])+2*sum_res_par+4*sum_res_impar)})

HTML widgets cannot be represented in plain text (need html)

<img src="https://dl.dropboxusercontent.com/s/ka6b4ewoum3xzgt/profvis_prog1.png?dl=0" heigth="500" width="1500">


**Nota:** la imagen anterior es un screenshot que generé ejecutando la celda anterior. 

En ese output podemos observar que se tarda mucho la función en las líneas de las sumas:
$$sum\_res\_par<-sum(f(nodes[is.even(seq\_along(nodes))]))$$ y $$sum\_res\_impar<-sum(f(nodes[is.odd(seq\_along(nodes))]))$$
   
Por lo que, esas dos líneas son las que se tienen que mejorar.

Para ello, se sustituyen las líneas anteriores por ciclos for en una versión mejorada

**Opción 2: cálculo de sumas con ciclos for**

In [24]:
n<-10**6 #cambiamos el número de subintervalos para ver su ejecución

In [28]:
Scf2<-function(n,f,a,b){
    #Compute numerical approximation using composed Simpson rule in 
    #an interval.
    #Nodes are generated via linspace
    #Args:
       # f (function): function of integrand
       # a (int): left point of interval
       # b (int): right point of interval
       # n (int): number of subintervals
    #Returns:
        #Scf2 (float)
   h<-(b-a)
   nodes1<-linspace(a,b,n+1)
   nodes<-nodes1[2:(length(nodes1)-1)]
   sum_res_par<-0
   sum_res_impar<-0
   for (i in 1:(n/2-1)){
      sum_res_par<-sum_res_par+f(nodes[2*i])
   }
   for (i in 1:(n/2)){
      sum_res_impar<-sum_res_impar+f(nodes[2*i-1])
   }
   h/(3*n)*(f(nodes1[1])+f(nodes1[length(nodes1)])+2*sum_res_par+4*sum_res_impar)
}

In [29]:
system.time(aprox<-Scf2(n,f,a,b))

   user  system elapsed 
  0.493   0.015   0.518 

Con esto se puede observar que el tiempo usado por el CPU fue de aprox .49 segundos y el tiempo en reloj fue de .51 segundos. Con lo que se redujeron los tiempos de forma drástica.

Posteriormente, verificamos que el error relativo se mantiene estable:

In [30]:
err_relativo(aprox,obj$value)

[1] 6.392347e-15

In [31]:
tic("medición de sleep + regla de Simpson")
tic("medición de tiempo de regla de Simpson con tictoc")
tic()
Scf2(n,f,a,b)
toc()
Sys.sleep(1)
toc()

[1] 0.7468241

elapsed time is 0.511000 seconds 
elapsed time is 1.516000 seconds 


In [32]:
profvis({h<-(b-a)
   nodes1<-linspace(a,b,n+1)
   nodes<-nodes1[2:(length(nodes1)-1)]
   sum_res_par<-0
   sum_res_impar<-0
   for (i in 1:(n/2-1)){
      sum_res_par<-sum_res_par+f(nodes[2*i])
   }
   for (i in 1:(n/2)){
      sum_res_impar<-sum_res_impar+f(nodes[2*i-1])
   }
   h/(3*n)*(f(nodes1[1])+f(nodes1[length(nodes1)])+2*sum_res_par+4*sum_res_impar)})

HTML widgets cannot be represented in plain text (need html)

<img src="https://dl.dropboxusercontent.com/s/tjesesntp747t61/profvis_prog2.png?dl=0" heigth="500" width="1500">


**Nota:** la imagen anterior es un screenshot que generé ejecutando la celda anterior. 

En ese output podemos observar que el tiempo de ejecución y de memoria para la función bajó considerablemente a pesar de que se están utilizando ciclos. Cabe señalar que el error relativo en esta segunda opción es mayor (15 vs 16 dígitos de precisión tomando más subintervalos en este caso), pero esto puede deberse a la aritmética de máquina dado que en cada una de las sumas se pueden estar perdiendo dígitos significativos, cosa que no pasa en la primera opción porque a cada entrada le calcula f(x) y despúes hace la suma de todas esas entradas.

**Opción 3: usando vapply**

Finalmente, intentaremos mejorar el cálculo de las sumas utilizando la función de vapply pues el tamaño de los vectores que alojan los nodos está bien definido:

In [18]:
Scf3<-function(n,f,a,b){
    #Compute numerical approximation using composed Simpson rule in 
    #an interval.
    #Nodes are generated via linspace
    #Args:
       # f (function): function of integrand
       # a (int): left point of interval
       # b (int): right point of interval
       # n (int): number of subintervals
    #Returns:
        #Scf3 (float)
   h<-(b-a)
   nodes1<-linspace(a,b,n+1)
   nodes<-nodes1[2:(length(nodes1)-1)]
   sum_res_par<-sum(f(nodes[vapply(seq_along(nodes), function(i) is.even(i),logical(1))]))
   sum_res_impar<-sum(f(nodes[vapply(seq_along(nodes), function(i) is.odd(i),logical(1))]))
   h/(3*n)*(f(nodes1[1])+f(nodes1[length(nodes1)])+2*sum_res_par+4*sum_res_impar)
}

In [19]:
system.time(aprox<-Scf3(n,f,a,b))

   user  system elapsed 
  0.335   0.000   0.336 

Con esto se puede observar que el tiempo usado por el CPU fue aprox. el mismo que el tiempo en reloj. Además vuelve a mejorar respecto a la versión anterior

Adicionalmente, encontramos un error relativo más bajo:

In [20]:
err_relativo(aprox,obj$value)

[1] 2.973185e-16

In [21]:
tic("medición de sleep + regla de Simpson")
tic("medición de tiempo de regla de Simpson con tictoc")
tic()
Scf3(n,f,a,b)
toc()
Sys.sleep(1)
toc()

[1] 0.7468241

elapsed time is 0.346000 seconds 
elapsed time is 1.351000 seconds 


In [22]:
profvis({h<-(b-a)
   nodes1<-linspace(a,b,n+1)
   nodes<-nodes1[2:(length(nodes1)-1)]
   sum_res_par<-sum(f(nodes[vapply(seq_along(nodes), function(i) is.even(i),logical(1))]))
   sum_res_impar<-sum(f(nodes[vapply(seq_along(nodes), function(i) is.odd(i),logical(1))]))
   h/(3*n)*(f(nodes1[1])+f(nodes1[length(nodes1)])+2*sum_res_par+4*sum_res_impar)})

HTML widgets cannot be represented in plain text (need html)

<img src="https://dl.dropboxusercontent.com/s/ev7qgkz4k64t0d0/profvis_prog3.png?dl=0" heigth="500" width="1500">


**Nota:** la imagen anterior es un screenshot que generé ejecutando la celda anterior. 

En el output final podemos observar que el tiempo de ejecución y de memoria para la función bajó significativamente y aún así conservamos 16 dígitos de precisión con el mismo número de subintervalos de la opción 2. Además, encontramos que la implementación de funciones como vapply que tienen un output específico (en este caso vectores) agilizan el cálculo de las sumas debido al tipo de objeto y tamaño bien definido.